# DEMO sprint 2 - Deel 2 (belastingen)
Inlezen waterstanden uit verschillende bronsystemen:
- RWS Waterinfo
- FEWS
- (Noos) Matroos
- RWS WaterWebServices

Gebruikmakend van:
- een demo omgeving van Continu Inzicht WebViewer (data in een Continu Inzicht PostGreSQL database)

### Benodigde generieke Python packages

In [1]:
from pathlib import Path

### Definitie configuratie: Database bestanden 

GlobalVariables:
    rootdir: "data_demo" 
    moments: [-24,0,24,48]  

    LoadsWaterinfo:
        parameters: ["waterhoogte"]
        MISSING_VALUE: 999

    LoadsFews:
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "HHNK_WEB"
        parameters: ["WATHTE [m][NAP][OW]"]
        MISSING_VALUE: 999
    
    LoadsMatroos:
        website: "vitaal"
        model: "dcsm6_kf"
        parameters: ["WATHTE"]
        MISSING_VALUE: 999
    
    LoadsWaterwebservicesRWS:
        parameters: ["WATHTE"]
        MISSING_VALUE: 999

DataAdapter: 
    BelastingLocaties_WaterInfo:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"        
        source: "WaterInfo"
    Waterstanden_WaterInfo:
        type: ci_postgresql_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"
        objecttype: "measuringstation" 
    BelastingLocaties_FEWS:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"        
        source: "FEWS"
    Waterstanden_FEWS:
        type: ci_postgresql_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"
        objecttype: "measuringstation"
    BelastingLocaties_Matroos:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"        
        source: "NOOS Matroos"
    Waterstanden_Matroos:
        type: ci_postgresql_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"
        objecttype: "measuringstation"
    BelastingLocaties_Waterwebservices_RWS:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"        
        source: "RWS Waterwebservices"
    Waterstanden_Waterwebservices_RWS:
        type: ci_postgresql_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_whatif"
        objecttype: "measuringstation"

### Lees configuratie in en maak dictionary base.Config aan

In [2]:
from toolbox_continu_inzicht import Config

yaml_config_file = "demo_sprint2-II.yaml"
path = Path.joinpath(Path.cwd(), "data_demo", yaml_config_file)
c = Config(config_path=path)
c.lees_config()

### Geef config mee aan base.Adapter

In [3]:
from toolbox_continu_inzicht import DataAdapter

data = DataAdapter(config=c)
print(f"Folder met data: {data.config.config_path}")
print(f"Globale variabelen: {data.config.global_variables}")
for data_adapter in data.config.data_adapters:
    print(f"Data apdater: {data_adapter}")

Folder met data: c:\Data\Python\toolbox-continu-inzicht\tests\demos\loads\data_demo\demo_sprint2-II.yaml
Globale variabelen: {'rootdir': 'data_demo', 'moments': [-24, 0, 24, 48], 'LoadsWaterinfo': {'parameters': ['waterhoogte'], 'MISSING_VALUE': 999}, 'LoadsFews': {'host': 'https://fews.hhnk.nl', 'port': 443, 'region': 'fewspiservice', 'version': '1.25', 'filter': 'HHNK_WEB', 'parameters': ['WATHTE [m][NAP][OW]'], 'MISSING_VALUE': 999}, 'LoadsMatroos': {'website': 'vitaal', 'model': 'dcsm6_kf', 'parameters': ['WATHTE'], 'MISSING_VALUE': 999}, 'LoadsWaterwebservicesRWS': {'parameters': ['WATHTE'], 'MISSING_VALUE': 999}}
Data apdater: BelastingLocaties_WaterInfo
Data apdater: Waterstanden_WaterInfo
Data apdater: BelastingLocaties_FEWS
Data apdater: Waterstanden_FEWS
Data apdater: BelastingLocaties_Matroos
Data apdater: Waterstanden_Matroos
Data apdater: BelastingLocaties_Waterwebservices_RWS
Data apdater: Waterstanden_Waterwebservices_RWS


### Invoerbestand (CSV) met data van belastinglocaties
Verplichte velden (schema):
- measurement_location_id: int64
- measurement_location_code: object (string)
- measurement_location_description: object (string)

![image info](data_demo/images/Database_measuringstations.PNG)

### Laad module (functie) LoadsWaterinfo voor inlezen belastingen uit RWS Waterinfo

In [4]:
from toolbox_continu_inzicht.loads import LoadsWaterinfo

waterinfo = LoadsWaterinfo(data_adapter=data)

### Run module (functie) LoadsWaterinfo voor inlezen waterstanden uit Waterinfo
LET OP: zoek op basis van een measurement_location_code die in RWS WaterInfo overeenkomt met 'locationCodes'!\
https://waterinfo.rws.nl/api/chart/get?mapType=waterhoogte&locationCodes=Pannerdense-kop(PANN)&values=-48%2C48

In [5]:
ds = waterinfo.run(input="BelastingLocaties_WaterInfo", output="Waterstanden_WaterInfo")

In [6]:
measuringlocations = waterinfo.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description
0,1,Pannerdense-kop(PANN),Pannerdense kop


In [7]:
waterlevels = waterinfo.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,parameter_description,unit,date_time,value,value_type,objecttype,calculating,datetime,parameterid
0,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 09:50:00+00:00,7.72,meting,measuringstation,False,1.731319e+12,1
1,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 10:00:00+00:00,7.71,meting,measuringstation,False,1.731319e+12,1
2,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 10:10:00+00:00,7.72,meting,measuringstation,False,1.731320e+12,1
3,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 10:20:00+00:00,7.73,meting,measuringstation,False,1.731320e+12,1
4,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 10:30:00+00:00,7.72,meting,measuringstation,False,1.731321e+12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:20:00+00:00,7.57,verwachting,measuringstation,False,1.731648e+12,2
572,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:30:00+00:00,7.57,verwachting,measuringstation,False,1.731649e+12,2
573,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:40:00+00:00,7.57,verwachting,measuringstation,False,1.731649e+12,2
574,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:50:00+00:00,7.57,verwachting,measuringstation,False,1.731650e+12,2


### Laad en run module (functie) LoadsFEWS voor inlezen belastingen uit FEWS
LET OP: zoek op basis van een measurement_location_code die in FEWS overeenkomt met 'locationId'!\
https://fews.hhnk.nl/FewsWebServices/rest/fewspiservice/v1/locations\
![image info](data_demo/images/FEWS_locationIs.PNG)

In [8]:
from toolbox_continu_inzicht.loads import LoadsFews

fews = LoadsFews(data_adapter=data)
ds = fews.run(input="BelastingLocaties_FEWS", output="Waterstanden_FEWS")

In [9]:
measuringlocations = fews.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description
0,2,MPN-N-24,Meetpunt Spijkerboor


In [10]:
waterlevels = fews.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,parameter_description,unit,date_time,value,value_type,objecttype,calculating,datetime,parameterid
0,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:00:00+00:00,-0.00463,meting,measuringstation,False,1.731402e+12,1
1,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:15:00+00:00,-0.00463,meting,measuringstation,False,1.731403e+12,1
2,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:30:00+00:00,-0.00475,meting,measuringstation,False,1.731404e+12,1
3,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:45:00+00:00,-0.00482,meting,measuringstation,False,1.731405e+12,1
4,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 10:00:00+00:00,-0.00472,meting,measuringstation,False,1.731406e+12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 08:45:00+00:00,-0.00482,meting,measuringstation,False,1.731488e+12,1
96,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 09:00:00+00:00,-0.00474,meting,measuringstation,False,1.731488e+12,1
97,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 09:15:00+00:00,-0.00466,verwachting,measuringstation,False,1.731489e+12,2
98,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 09:30:00+00:00,-0.00488,verwachting,measuringstation,False,1.731490e+12,2


### Laad en run module (functie) LoadsMatroos voor inlezen belastingen uit Matroos
LET OP: zoek op basis van een measurement_location_code die in (NOOS) Matroos overeenkomt met 'Location' -> loc_id!\
https://vitaal.matroos.rws.nl/matroos/php/image_series.php?type=image&source_id=156&loc_id=7&unit_id=1&colors=blue&ymin=0&ymax=0&label=0&alarm=0&alert=0&prealert=0&localtime_offset=0&tcurrent=2024-11-11%2019:38:11&tstart=2024-11-10%2000:00:00&tstop=2024-11-13%2000:00:00&anal_time=2024-11-11%2000:00:00

In [11]:
from toolbox_continu_inzicht.loads import LoadsMatroos

matroos = LoadsMatroos(data_adapter=data)
ds = matroos.run(input="BelastingLocaties_Matroos", output="Waterstanden_Matroos")

In [12]:
measuringlocations = matroos.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description
0,3,hoekvanholland,Hoek van Holland


In [13]:
waterlevels = matroos.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,date_time,unit,value,value_type,objecttype,calculating,datetime,parameterid
0,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:00:00+00:00,cm,0.0395,meting,measuringstation,False,1.731402e+12,1
1,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:10:00+00:00,cm,0.1130,meting,measuringstation,False,1.731403e+12,1
2,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:20:00+00:00,cm,0.1950,meting,measuringstation,False,1.731403e+12,1
3,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:30:00+00:00,cm,0.2961,meting,measuringstation,False,1.731404e+12,1
4,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:40:00+00:00,cm,0.3941,meting,measuringstation,False,1.731404e+12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:20:00+00:00,cm,-0.4088,verwachting,measuringstation,False,1.731648e+12,2
411,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:30:00+00:00,cm,-0.4246,verwachting,measuringstation,False,1.731649e+12,2
412,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:40:00+00:00,cm,-0.4199,verwachting,measuringstation,False,1.731649e+12,2
413,3,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:50:00+00:00,cm,-0.3954,verwachting,measuringstation,False,1.731650e+12,2


### Laad en run module (functie) LoadsWaterwebservicesRWS voor inlezen belastingen uit RWS Waterwebservices
LET OP: zoek op basis van een measurement_location_code die in RWS WaterWebservices overeenkomt met Locatie_MessageID!\
![image info](data_demo/images/FEWS_locationIs.PNG)

In [14]:
from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

waterwebservices_rws = LoadsWaterwebservicesRWS(data_adapter=data)
ds = waterwebservices_rws.run(
    input="BelastingLocaties_Waterwebservices_RWS",
    output="Waterstanden_Waterwebservices_RWS",
)

In [15]:
measuringlocations = waterwebservices_rws.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description
0,4,9889,Hintham beneden


### Resultaten in Continu Inzicht Webviewer
https://tscontinuinzicht.hkvservices.nl/ci-demo\
![image info](data_demo/images/Database_data_waterlevels.PNG)

<!-- values_nan = df_out[df_out["value"] == -999].index
for val in values_nan:
    df_out.loc[val, "value"] = np.nan
df_out.sort_index()
df_plot = df_out.set_index(df_out["datetime"])
df_plot = df_plot[df_plot["measurement_location_id"] == 8420]
measurement_index = df_plot["value_type"] == "verwachting"
df_plot_measurements = df_plot[measurement_index][["value"]]
df_plot_forecast = df_plot[~measurement_index][["value"]]
fig, ax = plt.subplots()
df_plot_measurements.plot(color="C0", ax=ax)
df_plot_forecast.plot(color="C1", ax=ax)
ax.legend(["Prediction", "Measurement"]) -->